
# Relatório Final : Maratona de Filmes

Aluno: Davi Reis Vieira de Souza

## Introdução

Queremos passar um final de semana assistindo ao máximo de filmes possível, mas há restrições quanto aos horários disponíveis e ao número de títulos que podem ser vistos em cada categoria (comédia, drama, ação, etc).

**Entrada**: Um inteiro N representando o número de filmes disponíveis para assistir e N trios de inteiros (H[i], F[i], C[i]), representando a hora de início, a hora de fim e a categoria do i-ésimo filme. Além disso, um inteiro M representando o número de categorias e uma lista de M inteiros representando o número máximo de filmes que podem ser assistidos em cada categoria.

**Saída**: Um inteiro representando o número máximo de filmes que podem ser assistidos de acordo com as restrições de horários e número máximo por categoria.

Abaixo, temos o seguinte exemplo de *input*:

```txt
10 4
1 3 1 2 
11 13 3
14 15 3
10 16 2
```

Como ler esse arquivo?

- A primeira linha indica que há 10 filmes a serem considerados e 4 categorias;
- a segunda linha indica qual o máximo de filmes que cada categoria pode ter;
- da terceira linha em diante você vai encontrar os n filmes, suas respectivas hora de início, hora de término e categoria pertencente.

## Objetivo

O objetivo do projeto é conseguir agrupar o máximo de filmes que podem ser assistidos em um dia conforme as instruções e as instruções dadas.

## Paralelismo com OpenMP

### Paralelismo com OpenMP

Até aquele momento, haviam sido experimentadas heurísticas que visavam resolver o problema em um tempo razoável, sem oferecer garantias de otimalidade. Agora, surge a necessidade de incorporar o paralelismo de tarefas nas alternativas de resolução.

Com o objetivo de introduzir o paralelismo, a implementação exaustiva precisará ser modificada. A diretiva ``#pragma omp parallel for`` pode ser utilizada para distribuir as iterações de um loop entre as threads disponíveis. Dentro desse loop, cada filme pode ser verificado individualmente e, caso esteja dentro das restrições de horário e categoria, uma variável compartilhada chamada ``"count"`` será incrementada. É importante ressaltar que, por se tratar de uma variável compartilhada, a região crítica precisa ser preservada entre as threads.

É relevante destacar que a utilização do OpenMP não necessariamente garantirá um melhor desempenho, pois a paralelização implica em um overhead que pode diminuir o desempenho do programa em alguns casos. É fundamental realizar testes para verificar se a utilização do OpenMP é verdadeiramente benéfica para o problema em questão.

## O Código

### Paralelismo com OpenMP

Abaixo, a explicação por partes principais do código:

#### Bibliotecas

```cpp
#include <iostream>
#include <vector>
#include <algorithm>
#include <iomanip>
#include <bitset>

#include <cmath>
#include <stack>
#include <omp.h>
```

São as bibliotecas padrões da linguagem C++. `iostream` é para entrada e saída de dados, ``vector`` é para armazenar listas de objetos, ``algorithm`` é para algoritmos de ordenação e outras operações em contêineres, ``iomanip`` é para manipulação de fluxos de saída e ``bitset`` é para manipulação de bits. ``cmath`` é para funções matemáticas, ``stack`` é para pilhas e ``omp.h`` é para paralelismo com OpenMP.

#### Estruturas de Dados

```cpp
struct Filme {
    int id;
    int inicio;
    int fim;
    int categoria;
    std::bitset<24> horario;
};

struct Categoria {
    int id;
    int quantidade;
};

struct Maratona {
    std::bitset<24> disponibilidade;
    std::vector<Filme> filmes;
};
```

São as estruturas de dados utilizadas no código. A estrutura ``Filme`` armazena as informações de cada filme, como id, horário de início e fim e categoria. A estrutura ``Categoria`` armazena o id e a quantidade de filmes de cada categoria. A estrutura ``Maratona`` armazena a disponibilidade de horários da maratona e os filmes selecionados.

#### Funções Auxiliares

```cpp
std::bitset<24> gera_horario(int inicio, int fim) {
    std::bitset<24> horario;

    if (inicio == fim) {
        horario.set(inicio);
        return horario;
    }
    
    for (int i = inicio; i < fim; i++) {
        horario.set(i);
    }

    return horario;
}
```

A função ``gera_horario`` recebe o horário de início e fim de um filme e retorna um std::bitset<24> com os bits correspondentes ao horário ocupado pelo filme. 

#### Função Principal

```cpp
int main() {
    int n, m;
    cin >> n >> m; // lê o número de filmes e de categorias
    vector<Filme> filmes(n); // cria um vetor de filmes com tamanho n
    vector<Categoria> categorias(m); // cria um vetor de categorias com tamanho m
    Maratona maratona; // cria uma maratona

    // Lê a quantidade de filmes de cada categoria
    for (int i = 0; i < m; i++) {
        cin >> categorias[i].quantidade;
        categorias[i].id = i + 1;
    }

    // Lê as informações dos filmes
    for (int i = 0; i < n; i++) {
        int inicio, fim, categoria;
        cin >> inicio >> fim >> categoria;

        // Se o horário de início é maior que o horário de fim,
        // significa que o filme se estende até o dia seguinte
        if (inicio > fim) {
            if (fim == 0) {
                fim = 24;
            } else if (inicio == -1 || fim == -1) {
                continue; // Ignora filmes com horários inválidos
            } else {
                continue; // Ignora filmes com horários inválidos
            }
        }

        // Cria um filme com as informações lidas
        Filme filme;
        filme.id = i + 1;
        filme.inicio = inicio;
        filme.fim = fim;
        filme.categoria = categoria;
        filme.horario = gera_horario(inicio, fim);

        // Adiciona o filme ao vetor de filmes
        filmes.push_back(filme);
    }

    // Chamada da função de busca exaustiva
    busca_exaustiva(filmes, categorias, maratona);

    return 0;
}
```

#### Busca Exaustiva

```cpp
void busca_exaustiva(vector<Filme> &filmes, vector<Categoria> &categorias, Maratona &maratona)
{
    int maximo = 0;
    int size_of_filmes = filmes.size();
    cout << "Quantidade total de filmes: " << size_of_filmes << endl;

    long int todas_combinacoes = pow(2, size_of_filmes);
    cout << "Quantidade total de combinações: " << todas_combinacoes << endl;
    long int i;

#pragma omp parallel for
    for (i = 0; i < todas_combinacoes; i++)
    {
        int num_films = 0;
        vector<int> categorias_vistas(categorias.size(), 0);
        stack<Filme> filmes_vistos;
        Maratona maratona_atual;

        bitset<64> filmes_vector(i);
        bitset<64> aux = filmes_vector;

        for (int j = 0; j < size_of_filmes; j++)
        {
            if (aux[j] == 1)
            {
                if ((maratona_atual.disponibilidade & filmes[j].horario) == 0)
                {
                    num_films++;
                    filmes_vistos.push(filmes[j]);
                    maratona_atual.disponibilidade |= filmes[j].horario;
                    categorias_vistas[filmes[j].categoria - 1]++;
                }

                aux[j] = 0;
            }
        }

#pragma omp critical
        if (num_films > maximo)
        {
            maximo = num_films;
            maratona.filmes.clear();
            maratona.disponibilidade.reset();

            while (!filmes_vistos.empty())
            {
                Filme filme = filmes_vistos.top();
                filmes_vistos.pop();
                maratona.filmes.push_back(filme);
                maratona.disponibilidade |= filme.horario;
            }
        }
    }

    cout << "Quantidade máxima possível: " << maximo << endl;

    for (int i = 0; i < maximo; i++)
    {
        cout << "Filme: [" << maratona.filmes[i].id << "] " << maratona.filmes[i].inicio << " " << maratona.filmes[i].fim << " " << maratona.filmes[i].categoria << endl;
    }
}
```

A função ``busca_exaustiva`` é responsável por realizar a busca exaustiva para encontrar a melhor combinação de filmes em uma maratona. Vamos analisar passo a passo o que essa função faz:

1. A função recebe como parâmetros um vetor de filmes (``filmes``), um vetor de categorias (``categorias``) e um objeto Maratona vazio (``maratona``).

2. É declarada uma variável ``maximo`` para armazenar o número máximo de filmes que podem ser incluídos na maratona.

3. O tamanho do vetor ``filmes`` é armazenado na variável ``size_of_filmes``.

4. É exibida na tela a quantidade total de filmes.

5. É calculado o número total de combinações possíveis usando a fórmula 2^N, onde N é o número de filmes. Essa quantidade é armazenada na variável ``todas_combinacoes``.

6. É exibida na tela a quantidade total de combinações.

7. Inicia-se um loop paralelo usando a diretiva ``#pragma omp parallel for``, que itera sobre todas as combinações possíveis.

8. Dentro do loop, são declaradas variáveis locais para contar o número de filmes selecionados (``num_films``), um vetor para registrar quantas categorias foram vistas (``categorias_vistas``), uma pilha de filmes selecionados (``filmes_vistos``) e um objeto Maratona temporário (``maratona_atual``).

9. Um objeto ``bitset`` chamado ``filmes_vector`` é criado a partir do número da iteração atual (``i``). Esse bitset é utilizado para determinar quais filmes serão selecionados na combinação atual.

10. Um loop ``for`` itera sobre os filmes do vetor ``filmes``.

11. Dentro do loop, é verificado se o bit correspondente em ``filmes_vector`` é igual a 1. Se for, significa que esse filme deve ser selecionado.

12. É verificado se o horário do filme não conflita com os filmes já selecionados na maratona atual. Isso é feito comparando o bitset de horário do filme com o bitset de disponibilidade da maratona atual (``maratona_atual.disponibilidade``). Se não houver conflito, o filme é considerado válido.

13. Se o filme for válido, incrementa-se o número de filmes selecionados (``num_films``), adiciona-se o filme à pilha ``filmes_vistos``, atualiza-se o bitset de disponibilidade da maratona atual e registra-se que uma categoria foi vista incrementando o valor correspondente no vetor ``categorias_vistas``.

14. O bit correspondente em ``aux`` é setado como 0, indicando que esse filme já foi selecionado.

15. Após o loop, é verificado se o número de filmes selecionados (``num_films``) é maior que o número máximo encontrado até o momento (``maximo``). Se for, significa que essa combinação de filmes é a melhor até agora, então o número máximo é atualizado, e as informações da maratona atual são copiadas para o objeto ``maratona``.

16. Após o loop paralelo, o número máximo possível de filmes é exibido na tela.

17. É exibida na tela a lista de filmes selecionados na maratona, com o ID, horário de início, horário de fim e categoria de cada filme.

Essa função utiliza paralelismo para acelerar a busca exaustiva, dividindo as combinações entre diferentes threads. Cada thread avalia uma parte das combinações e, ao final, o resultado é combinado para obter a melhor maratona possível.

## Arquivo Output

O arquivo de saída é gerado pelo programa e contém a:

- Quantidade de filmes na maratona
- Lista de filmes, com seus respectivos:
    - IDs
    - Horário de início
    - Horário de fim
    - Categoria

Exemplo de arquivo de saída:

```
20
474 0 1 20
99 1 2 70
184 2 4 12
581 4 5 52
92 5 6 9
849 6 7 18
746 7 8 33
319 8 9 3
179 9 10 74
207 10 11 57
757 11 12 18
618 13 13 17
628 14 15 36
358 16 16 87
351 17 18 35
304 18 19 87
794 19 20 44
51 20 21 54
360 22 22 37
541 23 24 39
```

Desta forma, é possível facilmente pegar os dados de saída e gerar um arquivo de texto com a lista de filmes da maratona e, com isso, gerar gráficos a partir dos dados, como veremos mais a frente.

## Valgrind

Utilizando a ferramenta Valgrind, é possível analisar o programa e verificar se ele possui algum erro de memória. Além disso, é possível observar possíveis otimizações no código que podem ser feitas para melhorar a performance do programa.

Para o contexto deste Projeto, utilizaremos esta ferramenta para comparar a performace de cada estratégia de geração de maratonas.

Antes de continuar, é importante instalar a ferramenta Valgrind. Para isso, basta executar o seguinte comando no terminal:


In [ ]:
!sudo apt-get install valgrind

### Heurística Gulosa

In [ ]:
!valgrind --tool=callgrind --vgdb=no ./exaustiva < inputs/input-12.txt  > exaustiva_output_valgrind.txt

```bash

--vgdb=no ./exaustiva < inputs/input-12.txt  > exaustiva_output_valgrind.txt
==31063== Callgrind, a call-graph generating cache profiler
==31063== Copyright (C) 2002-2017, and GNU GPL'd, by Josef Weidendorfer et al.
==31063== Using Valgrind-3.18.1 and LibVEX; rerun with -h for copyright info
==31063== Command: ./exaustiva
==31063==
==31063== For interactive control, run 'callgrind_control -h'.
==31063==
==31063== Events    : Ir
==31063== Collected : 17869305
==31063==
==31063== I   refs:      17,869,305
```

In [2]:
!callgrind_annotate callgrind.out.31063 exaustiva.cpp 

--------------------------------------------------------------------------------
Profile data file 'callgrind.out.31063' (creator: callgrind-3.18.1)
--------------------------------------------------------------------------------
I1 cache: 
D1 cache: 
LL cache: 
Timerange: Basic block 0 - 3454209
Trigger: Program termination
Profiled target:  ./exaustiva (PID 31063, part 1)
Events recorded:  Ir
Events shown:     Ir
Event sort order: Ir
Thresholds:       99
Include dirs:     
User annotated:   exaustiva.cpp
Auto-annotation:  on

--------------------------------------------------------------------------------
Ir                  
--------------------------------------------------------------------------------
17,869,305 (100.0%)  PROGRAM TOTALS

--------------------------------------------------------------------------------
Ir                  file:function
--------------------------------------------------------------------------------
1,668,839 ( 9.34%)  ???:0x00000000000208a0 [/usr/l

### Aleatoriedade 

In [ ]:
!valgrind --tool=callgrind --vgdb=no ./aleatoriedade < inputs/input-10k.txt  > aleatoriedade_output_valgrind.txt

In [ ]:
!callgrind_annotate callgrind.out.956164 aleatoriedade.cpp 

```txt
--------------------------------------------------------------------------------
Profile data file 'callgrind.out.956164' (creator: callgrind-3.15.0)
--------------------------------------------------------------------------------
I1 cache: 
D1 cache: 
LL cache: 
Timerange: Basic block 0 - 8990130
Trigger: Program termination
Profiled target:  ./aleatoriedade (PID 956164, part 1)
Events recorded:  Ir
Events shown:     Ir
Event sort order: Ir
Thresholds:       99
Include dirs:     
User annotated:   aleatoriedade.cpp
Auto-annotation:  off

--------------------------------------------------------------------------------
Ir         
--------------------------------------------------------------------------------
42,245,947  PROGRAM TOTALS

--------------------------------------------------------------------------------
Ir         file:function
--------------------------------------------------------------------------------
8,078,213  ???:std::istreambuf_iterator<char, std::char_traits<char> > std::num_get<char, std::istreambuf_iterator<char, std::char_traits<char> > >::_M_extract_int<long>(std::istreambuf_iterator<char, std::char_traits<char> >, std::istreambuf_iterator<char, std::char_traits<char> >, std::ios_base&, std::_Ios_Iostate&, long&) const [/usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.28]
5,462,182  /build/glibc-SzIz7B/glibc-2.31/libio/getc.c:getc [/usr/lib/x86_64-linux-gnu/libc-2.31.so]
4,225,106  /build/glibc-SzIz7B/glibc-2.31/libio/genops.c:_IO_sputbackc [/usr/lib/x86_64-linux-gnu/libc-2.31.so]
3,557,984  /build/glibc-SzIz7B/glibc-2.31/libio/ioungetc.c:ungetc [/usr/lib/x86_64-linux-gnu/libc-2.31.so]
2,528,568  ???:std::istream::sentry::sentry(std::istream&, bool) [/usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.28]
2,001,366  ???:__gnu_cxx::stdio_sync_filebuf<char, std::char_traits<char> >::underflow() [/usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.28]
1,655,610  ???:std::istream::operator>>(int&) [/usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.28]
1,173,978  ???:0x0000000000126e50 [/usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.28]
1,052,479  /build/glibc-SzIz7B/glibc-2.31/elf/dl-lookup.c:_dl_lookup_symbol_x [/usr/lib/x86_64-linux-gnu/ld-2.31.so]
  964,224  /build/glibc-SzIz7B/glibc-2.31/libio/iofflush.c:fflush [/usr/lib/x86_64-linux-gnu/libc-2.31.so]
  664,202  ???:0x0000000000125250 [/usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.28]
  661,862  /usr/include/c++/9/bits/predefined_ops.h:void std::__introsort_loop<__gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, long, __gnu_cxx::__ops::_Iter_comp_iter<bool (*)(Filme, Filme)> >(__gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, __gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, long, __gnu_cxx::__ops::_Iter_comp_iter<bool (*)(Filme, Filme)>)'2
  648,680  ???:__gnu_cxx::stdio_sync_filebuf<char, std::char_traits<char> >::uflow() [/usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.28]
  606,923  ???:0x00000000048e9130 [???]
  572,508  /build/glibc-SzIz7B/glibc-2.31/libio/libioP.h:fflush
  572,301  /build/glibc-SzIz7B/glibc-2.31/elf/dl-lookup.c:do_lookup_x [/usr/lib/x86_64-linux-gnu/ld-2.31.so]
  542,660  /build/glibc-SzIz7B/glibc-2.31/libio/fileops.c:_IO_file_sync@@GLIBC_2.2.5 [/usr/lib/x86_64-linux-gnu/libc-2.31.so]
  541,836  ???:0x0000000000126f00 [/usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.28]
  527,435  aleatoriedade.cpp:compara_filme(Filme, Filme) [/home/user/log-comp-davirvs/supercomputacao/projetos/maratona-filmes/aleatoriedade]
  508,020  /usr/include/c++/9/bits/move.h:void std::__introsort_loop<__gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, long, __gnu_cxx::__ops::_Iter_comp_iter<bool (*)(Filme, Filme)> >(__gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, __gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, long, __gnu_cxx::__ops::_Iter_comp_iter<bool (*)(Filme, Filme)>)'2
  451,935  ???:std::ostream::flush() [/usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.28]
  444,753  ???:0x00000000048e6880 [???]
  404,742  /usr/include/c++/9/bits/stl_algo.h:void std::__introsort_loop<__gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, long, __gnu_cxx::__ops::_Iter_comp_iter<bool (*)(Filme, Filme)> >(__gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, __gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, long, __gnu_cxx::__ops::_Iter_comp_iter<bool (*)(Filme, Filme)>)'2 [/home/user/log-comp-davirvs/supercomputacao/projetos/maratona-filmes/aleatoriedade]
  297,429  /usr/include/c++/9/bits/stl_tree.h:main
  283,539  aleatoriedade.cpp:main [/home/user/log-comp-davirvs/supercomputacao/projetos/maratona-filmes/aleatoriedade]
  275,569  /build/glibc-SzIz7B/glibc-2.31/elf/../sysdeps/x86_64/dl-machine.h:_dl_relocate_object
  270,490  /usr/include/c++/9/bits/stl_tree.h:aleatoriedade(std::vector<Categoria, std::allocator<Categoria> >&, Maratona&, std::map<int, std::vector<Filme, std::allocator<Filme> >, std::less<int>, std::allocator<std::pair<int const, std::vector<Filme, std::allocator<Filme> > > > >)
  197,806  /usr/include/c++/9/bitset:gera_horario(int, int)
  197,476  /usr/include/c++/9/bits/stl_uninitialized.h:void std::vector<Filme, std::allocator<Filme> >::_M_realloc_insert<Filme const&>(__gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, Filme const&)
  182,190  ???:std::locale::id::_M_id() const [/usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.28]
  177,338  /usr/include/c++/9/bits/predefined_ops.h:void std::__introsort_loop<__gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, long, __gnu_cxx::__ops::_Iter_comp_iter<bool (*)(Filme, Filme)> >(__gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, __gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, long, __gnu_cxx::__ops::_Iter_comp_iter<bool (*)(Filme, Filme)>)
  163,152  /build/glibc-SzIz7B/glibc-2.31/stdlib/random_r.c:srandom_r [/usr/lib/x86_64-linux-gnu/libc-2.31.so]
  162,635  aleatoriedade.cpp:gera_horario(int, int) [/home/user/log-comp-davirvs/supercomputacao/projetos/maratona-filmes/aleatoriedade]
  153,769  /usr/include/c++/9/bits/stl_vector.h:main
  129,904  /usr/include/c++/9/bits/stl_algo.h:main
  119,220  /build/glibc-SzIz7B/glibc-2.31/elf/dl-lookup.c:check_match [/usr/lib/x86_64-linux-gnu/ld-2.31.so]
  106,348  /usr/include/c++/9/bits/move.h:void std::__introsort_loop<__gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, long, __gnu_cxx::__ops::_Iter_comp_iter<bool (*)(Filme, Filme)> >(__gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, __gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, long, __gnu_cxx::__ops::_Iter_comp_iter<bool (*)(Filme, Filme)>)
   90,749  /build/glibc-SzIz7B/glibc-2.31/elf/do-rel.h:_dl_relocate_object
   90,387  ???:__gnu_cxx::stdio_sync_filebuf<char, std::char_traits<char> >::sync() [/usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.28]
   90,258  /usr/include/c++/9/bits/stl_algo.h:void std::__introsort_loop<__gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, long, __gnu_cxx::__ops::_Iter_comp_iter<bool (*)(Filme, Filme)> >(__gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, __gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, long, __gnu_cxx::__ops::_Iter_comp_iter<bool (*)(Filme, Filme)>) [/home/user/log-comp-davirvs/supercomputacao/projetos/maratona-filmes/aleatoriedade]
   85,980  /build/glibc-SzIz7B/glibc-2.31/string/../sysdeps/x86_64/strcmp.S:strcmp [/usr/lib/x86_64-linux-gnu/ld-2.31.so]
   85,707  /usr/include/c++/9/bits/predefined_ops.h:main
   71,545  /build/glibc-SzIz7B/glibc-2.31/elf/dl-addr.c:_dl_addr [/usr/lib/x86_64-linux-gnu/libc-2.31.so]
   60,607  ???:0x00000000048e64f0 [???]
   60,269  ???:0x00000000048e9080 [???]
   60,264  /build/glibc-SzIz7B/glibc-2.31/libio/../sysdeps/unix/sysv/linux/x86/lowlevellock.h:fflush
   60,215  ???:0x00000000048e7530 [???]
   60,209  ???:0x00000000048e6f30 [???]
   60,209  ???:0x00000000048e86a0 [???]
   60,204  ???:0x000000000010a2b0 [???]
   60,204  ???:std::num_get<char, std::istreambuf_iterator<char, std::char_traits<char> > >::do_get(std::istreambuf_iterator<char, std::char_traits<char> >, std::istreambuf_iterator<char, std::char_traits<char> >, std::ios_base&, std::_Ios_Iostate&, long&) const [/usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.28]
   50,214  /usr/include/c++/9/bits/stl_uninitialized.h:std::_Rb_tree_node<std::pair<int const, std::vector<Filme, std::allocator<Filme> > > >* std::_Rb_tree<int, std::pair<int const, std::vector<Filme, std::allocator<Filme> > >, std::_Select1st<std::pair<int const, std::vector<Filme, std::allocator<Filme> > > >, std::less<int>, std::allocator<std::pair<int const, std::vector<Filme, std::allocator<Filme> > > > >::_M_copy<std::_Rb_tree<int, std::pair<int const, std::vector<Filme, std::allocator<Filme> > >, std::_Select1st<std::pair<int const, std::vector<Filme, std::allocator<Filme> > > >, std::less<int>, std::allocator<std::pair<int const, std::vector<Filme, std::allocator<Filme> > > > >::_Alloc_node>(std::_Rb_tree_node<std::pair<int const, std::vector<Filme, std::allocator<Filme> > > > const*, std::_Rb_tree_node_base*, std::_Rb_tree<int, std::pair<int const, std::vector<Filme, std::allocator<Filme> > >, std::_Select1st<std::pair<int const, std::vector<Filme, std::allocator<Filme> > > >, std::less<int>, std::allocator<std::pair<int const, std::vector<Filme, std::allocator<Filme> > > > >::_Alloc_node&)'2
   46,897  /build/glibc-SzIz7B/glibc-2.31/malloc/malloc.c:_int_malloc [/usr/lib/x86_64-linux-gnu/libc-2.31.so]
   36,755  /usr/include/c++/9/bits/stl_map.h:main
   31,912  /usr/include/c++/9/bits/stl_map.h:aleatoriedade(std::vector<Categoria, std::allocator<Categoria> >&, Maratona&, std::map<int, std::vector<Filme, std::allocator<Filme> >, std::less<int>, std::allocator<std::pair<int const, std::vector<Filme, std::allocator<Filme> > > > >)
   28,881  /usr/include/c++/9/new:void std::vector<Filme, std::allocator<Filme> >::_M_realloc_insert<Filme const&>(__gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, Filme const&)
   27,973  /build/glibc-SzIz7B/glibc-2.31/malloc/malloc.c:_int_free [/usr/lib/x86_64-linux-gnu/libc-2.31.so]
   24,365  aleatoriedade.cpp:aleatoriedade(std::vector<Categoria, std::allocator<Categoria> >&, Maratona&, std::map<int, std::vector<Filme, std::allocator<Filme> >, std::less<int>, std::allocator<std::pair<int const, std::vector<Filme, std::allocator<Filme> > > > >) [/home/user/log-comp-davirvs/supercomputacao/projetos/maratona-filmes/aleatoriedade]
   20,916  /usr/include/c++/9/bits/stl_algo.h:void std::__move_median_to_first<__gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, __gnu_cxx::__ops::_Iter_comp_iter<bool (*)(Filme, Filme)> >(__gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, __gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, __gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, __gnu_cxx::__normal_iterator<Filme*, std::vector<Filme, std::allocator<Filme> > >, __gnu_cxx::__ops::_Iter_comp_iter<bool (*)(Filme, Filme)>) [/home/user/log-comp-davirvs/supercomputacao/projetos/maratona-filmes/aleatoriedade]

--------------------------------------------------------------------------------
-- User-annotated source: aleatoriedade.cpp
--------------------------------------------------------------------------------
Ir      

-- line 22 ----------------------------------------
      .      int quantidade;
      .  };
      .  
      .  struct Maratona {
      .      std::bitset<24> disponibilidade;
      .      std::vector<Filme> filmes;
      .  };
      .  
105,487  bool compara_filme(Filme a, Filme b) {
316,461      return a.fim < b.fim;
105,487  }
      .  
 18,172  std::bitset<24> gera_horario(int inicio, int fim) {
 18,042      std::bitset<24> horario;
      .  
 18,172      if (inicio == fim) {
     65          horario.set(inicio);
    195          return horario;
      .      }
      .      
 62,885      for (int i = inicio; i < fim; i++) {
 26,932          horario.set(i);
      .      }
      .  
      .      return horario;
 18,172  }
      .  
      .  
     14  void aleatoriedade(vector<Categoria> &categorias, Maratona &maratona, map<int, vector<Filme>> filmes_por_horario){
      1      unsigned seed = std::chrono::system_clock::now().time_since_epoch().count();
    992  => ???:0x000000000010a250 (1x)
      1      std::default_random_engine generator (seed);
      .      std::binomial_distribution<int> distribution (1, 0.75);
      .      distribution(generator);
      .  
      .      bitset<24> mochila_cheia(0xFFFFFF);
      .  
    123      for (int i = 1; i <= 24; i++){
     48          if (maratona.disponibilidade == mochila_cheia){
      .              break;
      .          }
      .  
     72          if (filmes_por_horario[i].size() == 0){
      .              continue;
      .          }
      .  
      .          Filme filme_escolhido;
      .  
     96          srand(time(NULL));
163,608  => ???:0x000000000010a360 (24x)
    192  => ???:0x000000000010a340 (24x)
     92          if (distribution(generator)){
  9,341              for (int j = 0; j < static_cast<int>(filmes_por_horario[i].size()); j++){
  6,438                  if ((!(maratona.disponibilidade & filmes_por_horario[i][j].horario).any()) && (categorias[filmes_por_horario[i][j].categoria - 1].quantidade > 0)){
    100                      filme_escolhido = filmes_por_horario[i][j];
     20                      break;
      .                  }
      .  
  3,099                  filme_escolhido.id = -1;
      .              }
      .          } else {
      .              vector<Filme> filmes_disponiveis_no_horario;
      .  
  2,485              for (int j = 1; j < static_cast<int>(filmes_por_horario[i].size()); j++){
  1,744                  if ((!(maratona.disponibilidade & filmes_por_horario[i][j].horario).any()) && (categorias[filmes_por_horario[i][j].categoria - 1].quantidade > 0)){
      .                      filmes_disponiveis_no_horario.push_back(filmes_por_horario[i][j]);
      .                  }
      .              }
      .  
      6              if (filmes_disponiveis_no_horario.size() == 0){
      .                  continue;
      .              }
      .  
     22              filme_escolhido = filmes_disponiveis_no_horario[rand() % filmes_disponiveis_no_horario.size()];
    124  => ???:0x000000000010a270 (2x)
      .          }
      .  
     48          if (filme_escolhido.id == -1){
      .              continue;
      .          }
      .  
      .          maratona.disponibilidade |= filme_escolhido.horario;
      6          maratona.filmes.push_back(filme_escolhido);
     88          categorias[filme_escolhido.categoria - 1].quantidade--;        
      .      }
      .  
      .      cout << maratona.filmes.size() << endl;
      .  
     70      for (int i = 0; i < static_cast<int>(maratona.filmes.size()); i++){
    440          cout << maratona.filmes[i].id << " " << maratona.filmes[i].inicio << " " << maratona.filmes[i].fim << " " << maratona.filmes[i].categoria << endl;
 38,891  => ???:0x000000000010a420 (88x)
      .      }
      .  
      .      cout << endl;
     11  }
      .  
      .  
     11  int main() {
      .      int n, m;
      6      cin >> n >> m;
  9,177  => ???:0x000000000010a2b0 (2x)
      .      vector<Filme> filmes;
      1      vector<Categoria> categorias(m);
      .      Maratona maratona;
      .      
    305      for (int i = 0; i < m; i++) {
    301          cin >> categorias[i].quantidade;
121,283  => ???:0x000000000010a2b0 (100x)
    300          categorias[i].id = i + 1;
      .      }
      .  
 20,005      for (int i = 0; i < n; i++) {
      .          int inicio, fim, categoria;
100,001          cin >> inicio >> fim >> categoria;
34,747,066  => ???:0x000000000010a2b0 (30,000x)
      .  
 50,000          if (inicio > fim) {
  2,666              if (fim == 0){
    838                  fim = 24;
      .              } else if (inicio == -1 || fim == -1) {
      .                  continue;
      .              } else {
      .                  continue;
      .              }
      .          }
      .  
      .          Filme filme;
  9,086          filme.id = i + 1;
  9,086          filme.inicio = inicio;
  9,086          filme.fim = fim;
 18,172          filme.categoria = categoria;
 18,172          filme.horario = gera_horario(inicio, fim);
360,441  => aleatoriedade.cpp:gera_horario(int, int) (9,086x)
      .  
      .          filmes.push_back(filme);
      .      }
      .  
      .      sort(filmes.begin(), filmes.end(), compara_filme);
      .  
      1      int size_of_filmes = filmes.size();
      .  
      .      // for (int i = 0; i < size_of_f    ilmes; i++) {
      .      //     cout << filmes[i].id << " " << filmes[i].inicio << " " << filmes[i].fim << endl;
      .      // }
      .  
      .      map<int, vector<Filme>> filmes_por_horario;
      .      
 45,436      for (int i = 0; i < size_of_filmes; i++) {
     50          filmes_por_horario[filmes[i].fim].push_back(filmes[i]);
      .      }
      .  
      4      aleatoriedade(categorias, maratona, filmes_por_horario);
605,510  => aleatoriedade.cpp:aleatoriedade(std::vector<Categoria, std::allocator<Categoria> >&, Maratona&, std::map<int, std::vector<Filme, std::allocator<Filme> >, std::less<int>, std::allocator<std::pair<int const, std::vector<Filme, std::allocator<Filme> > > > >) (1x)
      .  
      .      return 0;
      .  
     15  }

--------------------------------------------------------------------------------
Ir      
--------------------------------------------------------------------------------
997,977  events annotated
```

## Análise de desempenho

### Questões gerais sobre o desempenho dos programas

O output do *Valgrind* sugere que ambos os programas tem problemas de vazamento de memória. Esse erro ocorre provavelmente por conta da falta de desalocação de um recurso de memória.

Além disso, para a estratégia aleatória, outras indicações mostram um grande número de operações de alocação e liberação de memória em funções que são chamadas várias vezes no decorrer do programa, como std::chrono::system_clock::now().time_since_epoch().count(), std::default_random_engine generator, std::binomial_distribution<int> distribution e srand(time(NULL)), o que pode ser otimizado se for possível reduzir a quantidade de chamadas dessas funções.

Outra questão que pode ser melhorada é a alocação dinâmica de memória.


### Análise de desempenho do programa aleatório

Temos também, no modelo aleatório, um loop que serva para fazer uma lista apenas com filmes que estão disponíveis no horário atual. Esse consome bastante, apesar de ser um grande trade-off entre ficar apenas procurando um filme de forma aleatória, e é algo que pode ser melhorado, comparando mais casos.

### Análise de desempenho do programa guloso

O programa guloso possui muitas melhorias que podem ser feitas, como a alocação dinâmica de memória, a redução do número de alocações e liberações.

## Comparações com base em dados de desempenho

Antes, é importante realizar o entendimento a seguir:

### Arquivos de Input

Todos os arquivos de input possuem 5 categorias.

- **input-12.txt**: contém 12 filmes.
- **input-16.txt**: contém 16 filmes.
- **input-20.txt**: contém 20 filmes.
- **input-24.txt**: contém 24 filmes.
- **input-28.txt**: contém 28 filmes.
- **input-32.txt**: contém 32 filmes.

### Compilação

- g++ -Wl,-z,stack-size=33554432 -fopenmp exaustiva.cpp -o exaustiva
- g++ -Wall -O3 -g aleatoriedade.cpp -o aleatoriedade

### Execução

Exemplos: 
- ./exaustiva inputs/< input-1k.txt > outputs/output-exaustiva-12.txt
- ./aleatoriedade inputs/< input-1k.txt > outputs/output-aleatoriedade-1k.txt

- Importando bibliotecas

In [5]:
import time
import pandas as pd
import matplotlib.pyplot as plt

- Funções auxiliares

In [1]:
def number_of_movies(file):
    with open(file, 'r') as f:
        line = f.readline().split()
        return int(line[0])

- Criando uma estrutura de dados para armazenar os dados de cada execução

In [3]:
data = {
    'exaustiva' : {
        'input-12.txt': {
            'tempo': [],
            'memoria': []
        },
        'input-16.txt': {
            'tempo': [],
            'memoria': []
        },
        'input-20.txt': {
            'tempo': [],
            'memoria': []
        },
        'input-24.txt': {
            'tempo': [],
            'memoria': []
        },
        'input-28.txt': {
            'tempo': [],
            'memoria': []
        },
        'input-32.txt': {
            'tempo': [],
            'memoria': []
        },
    },
}

- Realizando o loop para cada arquivo de input e cada algoritmo

Abaixo, serão realizadas 10 interações para cada arquivo de input e cada algoritmo, e os resultados serão salvos em um arquivo de texto logo após.

In [6]:
for algoritmo in data:
    for arquivo in data[algoritmo]:
        for i in range(10):
            print(f'{algoritmo} - {arquivo} - {i+1}/10')
            output_name = str(i) + '_' + arquivo.split('-')[1]
            start_time = time.time()
            !./{algoritmo} < inputs/{arquivo} > outputs/{algoritmo}_output_{output_name}
            end_time = time.time() - start_time
            print('\tTime:', end_time)
            data[algoritmo][arquivo]['tempo'].append(end_time)
            data[algoritmo][arquivo]['memoria'].append(number_of_movies(f'outputs/{algoritmo}_output_{output_name}'))           

exaustiva - input-12.txt - 1/10
	Time: 0.16638565063476562
exaustiva - input-12.txt - 2/10
	Time: 0.1813032627105713
exaustiva - input-12.txt - 3/10
	Time: 0.17824220657348633
exaustiva - input-12.txt - 4/10
	Time: 0.17338156700134277
exaustiva - input-12.txt - 5/10
	Time: 0.18008708953857422
exaustiva - input-12.txt - 6/10
	Time: 0.18088126182556152
exaustiva - input-12.txt - 7/10
	Time: 0.17887067794799805
exaustiva - input-12.txt - 8/10
	Time: 0.17702770233154297
exaustiva - input-12.txt - 9/10
	Time: 0.1794140338897705
exaustiva - input-12.txt - 10/10
	Time: 0.16985392570495605
exaustiva - input-16.txt - 1/10
	Time: 0.16747760772705078
exaustiva - input-16.txt - 2/10
	Time: 0.1799936294555664
exaustiva - input-16.txt - 3/10
	Time: 0.1799774169921875
exaustiva - input-16.txt - 4/10
	Time: 0.1708226203918457
exaustiva - input-16.txt - 5/10
	Time: 0.1882164478302002
exaustiva - input-16.txt - 6/10
	Time: 0.17673921585083008
exaustiva - input-16.txt - 7/10
	Time: 0.17644119262695312
ex

- Salvando os resultados em um arquivo JSON para serem utilizados no gráfico em outro momento

In [ ]:
import json

with open('data.json', 'w') as fp:
    json.dump(json.dumps(data, indent=2), fp)

- Gerando o DataFrame para ser utilizado no gráfico

In [ ]:
df = pd.DataFrame(columns=['algoritmo', 'arquivo', 'tentativa', 'tempo', 'memoria'])

for algoritmo in data:
    for arquivo in data[algoritmo]:
        for i in range(10):
            df = pd.concat([df, pd.DataFrame({
                'algoritmo': [algoritmo],
                'arquivo': [arquivo],
                'tentativa': [i+1],
                'tempo': [data[algoritmo][arquivo]['tempo'][i]],
                'memoria': [data[algoritmo][arquivo]['memoria'][i]]
            })])

df

- Salvando o DataFrame em um arquivo CSV

In [ ]:
df.to_csv('data.csv', index=False)

In [ ]:
df.head(10)

### Média de Tempo por arquivo por algoritmo

In [ ]:
df_mean = df.groupby(['algoritmo', 'arquivo']).mean(numeric_only=True).reset_index()
df_mean

### Gráfico de tempo

In [ ]:
df_mean.sort_values(by=['tempo'], inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
for algoritmo in df_mean['algoritmo'].unique():
    df_aux = df_mean[df_mean['algoritmo'] == algoritmo]
    ax.plot(df_aux['arquivo'], df_aux['tempo'], label=algoritmo)

ax.set_title('Tempo por algoritmo por arquivo')
ax.set_xlabel('Arquivo')
ax.set_ylabel('Tempo (s)')
ax.legend()
plt.show()

Vemos que a estratégia aleatória possui uma vantagem maior quando temos um cenário de mais fimes no nosso arquivo de input.

Isso deve-se ao fato de que a estratégia gulosa percorre todos os filmes até completar a Maratona de Filmes, procurando filmes que possam ser colocados nela. Já a estratégia aleatória, ao invés de percorrer todos os filmes, escolhe aleatoriamente um filme que pode ser colocado na Maratona de Filmes e o insere.

- Criando o Dataset para o gráfico de Filmes na Mochila vs Input

In [ ]:
df_mean_movies = df.drop(columns=['tempo','tentativa'])
df_mean_movies = df_mean_movies.groupby(['algoritmo', 'arquivo']).mean().reset_index()
df_mean_movies

In [ ]:
order_arquivos = list(data['gulosa'].keys())

df_mean_movies['arquivo'] = pd.Categorical(df_mean_movies['arquivo'], categories=order_arquivos, ordered=True)
df_mean_movies = df_mean_movies.sort_values(by=['arquivo'])
df_mean_movies

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

for algoritmo in df_mean_movies['algoritmo'].unique():
    df_aux = df_mean_movies[df_mean_movies['algoritmo'] == algoritmo]
    ax.plot(df_aux['arquivo'], df_aux['memoria'], label=algoritmo)

ax.set_title('Filmes por algoritmo por arquivo')
ax.set_xlabel('Arquivo')
ax.set_ylabel('Filmes (número de filmes)')
ax.legend()
plt.show()


Já no gráfico de Filmes na Mochila vs Input, vemos que a estratégia gulosa possui uma vantagem maior quando temos um cenário de mais fimes no nosso arquivo de input. Isso deve-se ao fato de que a estratégia gulosa percorre todos os filmes até completar a Maratona de Filmes, procurando filmes que possam ser colocados nela. Já a estratégia aleatória, ao invés de percorrer todos os filmes, escolhe aleatoriamente um filme que pode ser colocado na Maratona de Filmes e o insere no bitset.

## Comparando categorias

Além disso, é possível comparar a quantidade de filmes por categoria que foram colocados na Maratona de Filmes.

Neste caso, serão utilizados 10000 filmes variando a quantidade de categorias de 1,2,3,4,5 e 10,20,30,50 e 100.

- **input-10k-1.txt**: contém 10000 filmes e 1 categoria.
- **input-10k-2.txt**: contém 10000 filmes e 2 categorias.
- **input-10k-3.txt**: contém 10000 filmes e 3 categorias.
- **input-10k-4.txt**: contém 10000 filmes e 4 categorias.
- **input-10k-5.txt**: contém 10000 filmes e 5 categorias.
- **input-10k-10.txt**: contém 10000 filmes e 10 categorias.
- **input-10k-20.txt**: contém 10000 filmes e 20 categorias.
- **input-10k-30.txt**: contém 10000 filmes e 30 categorias.
- **input-10k-50.txt**: contém 10000 filmes e 50 categorias.
- **input-10k-100.txt**: contém 10000 filmes e 100 categorias.

In [ ]:
data_categoria = {
    'gulosa' : {
        'input-10k-1.txt': {
            'temmpo': 0,
            'categoria': 0
        },
        'input-10k-2.txt': {
            'temmpo': 0,
            'categoria': 0
        },
        'input-10k-3.txt': {
            'temmpo': 0,
            'categoria': 0
        },
        'input-10k-4.txt': {
            'temmpo': 0,
            'categoria': 0
        },
        'input-10k-5.txt': {
            'temmpo': 0,
            'categoria': 0
        },
        'input-10k-10.txt': {
            'temmpo': 0,
            'categoria': 0
        },
        'input-10k-20.txt': {
            'temmpo': 0,
            'categoria': 0
        },
        'input-10k-30.txt': {
            'temmpo': 0,
            'categoria': 0
        },
        'input-10k-50.txt': {
            'temmpo': 0,
            'categoria': 0
        },
        'input-10k-75.txt': {
            'temmpo': 0,
            'categoria': 0
        },
        'input-10k-100.txt': {
            'temmpo': 0,
            'categoria': 0
        },
    },
    'aleatoriedade': {
        'input-10k-1.txt': {
            'temmpo': 0,
            'categoria': 0
        },
        'input-10k-2.txt': {
            'temmpo': 0,
            'categoria': 0
        },
        'input-10k-3.txt': {
            'temmpo': 0,
            'categoria': 0
        },
        'input-10k-4.txt': {
            'temmpo': 0,
            'categoria': 0
        },
        'input-10k-5.txt': {
            'temmpo': 0,
            'categoria': 0
        },
        'input-10k-10.txt': {
            'temmpo': 0,
            'categoria': 0
        },
        'input-10k-20.txt': {
            'temmpo': 0,
            'categoria': 0
        },
        'input-10k-30.txt': {
            'temmpo': 0,
            'categoria': 0
        },
        'input-10k-50.txt': {
            'temmpo': 0,
            'categoria': 0
        },
        'input-10k-75.txt': {
            'temmpo': 0,
            'categoria': 0
        },
        'input-10k-100.txt': {
            'temmpo': 0,
            'categoria': 0
        },
    }
}

In [ ]:
for algoritmo in data_categoria:
    for arquivo in data_categoria[algoritmo]:
        output_name = algoritmo + '_output_categoria_' + arquivo.split('-')[2]
        start_time = time.time()
        !./{algoritmo} < inputs/{arquivo} > outputs/{output_name}
        end_time = time.time() - start_time
        print('\tTime:', end_time)
        data_categoria[algoritmo][arquivo]['tempo'] = end_time
        data_categoria[algoritmo][arquivo]['memoria'] = number_of_movies(f'outputs/{output_name}')

In [ ]:
df_categoria = pd.DataFrame(columns=['algoritmo', 'arquivo', 'categoria', 'tempo', 'memoria'])

for algoritmo in data_categoria:
    for arquivo in data_categoria[algoritmo]:
        df_categoria = pd.concat([df_categoria, pd.DataFrame({
            'algoritmo': [algoritmo],
            'arquivo': [arquivo],
            'categoria': [int(arquivo.split('-')[2].split('.')[0])],
            'tempo': [data_categoria[algoritmo][arquivo]['tempo']],
            'memoria': [data_categoria[algoritmo][arquivo]['memoria']]
        })])

df_categoria

- Criando o Dataset para o gráfico de Tempo vs Categorias

In [ ]:
df_categoria_mean = df_categoria.drop(columns=['memoria'])
df_categoria_mean = df_categoria_mean.groupby(['algoritmo', 'arquivo', 'categoria']).mean().reset_index()
df_categoria_mean = df_categoria_mean.sort_values(by=['algoritmo','categoria'])
df_categoria_mean

In [ ]:
# plotar o tempo por categoria
fig, ax = plt.subplots(figsize=(10, 5))

for algoritmo in df_categoria_mean['algoritmo'].unique():
    df_aux = df_categoria_mean[df_categoria_mean['algoritmo'] == algoritmo]
    ax.plot(df_aux['categoria'], df_aux['tempo'], label=algoritmo)

ax.set_title('Tempo por algoritmo por categoria')
ax.set_xlabel('Categoria')
ax.set_ylabel('Tempo (s)')
ax.legend()
plt.show()

Ao tentar analisar o gráfico de Tempo vs Categorias, não se pode observar uma tendência clara, pois o tempo de execução é muito variável.

## Analisando filmes por categoria

- Funções auxiliares

In [ ]:
def get_all_filmes(arquivo, arquivo_name, algoritmo):
    with open(arquivo, 'r') as f:
        lines = f.readlines()
        filmes = []
        for line in lines:
            line_split = line.split(' ')
            if len(line_split) < 4:
                continue
            filme = {
                'id': int(line_split[0]),
                'inicio': int(line_split[1]),
                'fim': int(line_split[2]),
                'categoria': int(line_split[3].split('\n')[0]),
                'arquivo': arquivo_name,
                'algoritmo': algoritmo
            }
            filmes.append(filme)
        return filmes

- Gerando um dataset dos filmes por diferentes números de categorias

In [ ]:
df_filmes = pd.DataFrame(columns=['id', 'inicio', 'fim', 'categoria', 'arquivo', 'algoritmo'])

for algoritmo in data_categoria:
    for arquivo in data_categoria[algoritmo]:
        output_name = 'outputs/'+algoritmo + '_output_categoria_' + arquivo.split('-')[2]
        filmes = get_all_filmes(output_name, arquivo, algoritmo)
        df_filmes = pd.concat([df_filmes, pd.DataFrame(filmes)])

In [ ]:
df_filmes

- Salvar os dados em um arquivo csv


In [ ]:
df_filmes.to_csv('df_filmes_categoria.csv', index=False)

In [ ]:
# carrgar o arquivo
df_filmes = pd.read_csv('df_filmes_categoria.csv')
df_filmes

In [ ]:
df_filmes_count = df_filmes.drop(columns=['id', 'inicio', 'fim'])
df_filmes_count = df_filmes_count.groupby(['algoritmo', 'arquivo', 'categoria']).count().reset_index()
df_filmes_count = df_filmes_count.sort_values(by=['algoritmo','categoria'])
df_filmes_count

In [ ]:
# plotar grafico de dispersão

import numpy as np
fig, ax = plt.subplots(figsize=(20, 10))

for algoritmo in df_filmes_count['algoritmo'].unique():
    # plotar os pontos
    df_aux = df_filmes_count[df_filmes_count['algoritmo'] == algoritmo]
    ax.scatter(df_aux['categoria'], df_aux['arquivo'], label=algoritmo)


ax.set_title('Número de filmes por algoritmo por categoria')
ax.set_xlabel('Categoria')
ax.set_ylabel('Número de filmes')
ax.legend()
plt.show()


Com esta distribuição, pode-se ver que a quantidade de filmes por categoria vs quantidade de categorias pouco influencia na quantidade de filmes que serão colocados na Maratona de Filmes conforme o número de categorias aumenta. 

## Conclusões

Podemos conferir que, a partir destes resultados, não basta apenas escolher um algoritmo para resolver um problema, mas sim, analisar o cenário e escolher o algoritmo que melhor se encaixa.

Caso o problema seja assistir a maior quantidade de filmes, no caso da Maratona de Filmes, podemos utilizar algoritmos como o guloso, e que possui uma vantagem maior quando temos um cenário de mais fimes no nosso arquivo de input.

Caso o problema seja a otimização do tempo do algoritmo, podemos utilizar algoritmos como o aleatório, e que possui uma vantagem maior quando temos um cenário de mais fimes.

Além disso, com estes dados, conseguimos comparar o balanceamento de duas abordagens distintas: exploitation e exploration. No exploitation, o foco é dado a uma única propriedade determinística, que leva a soluções consistentes, porém, pode demandar mais tempo, neste caso. Por outro lado, no exploration, há liberdade para escolhas aleatórias, o que pode levar a soluções mais otimizadas em relação ao tempo, porém, com uma consistência no preenchimento total da Maratona menor.